In [1]:
import gym
import gym_watten
from gym_watten.envs.watten_env import Color, Value
import numpy as np
from time import sleep
import random

In [2]:
env = gym.make("Watten-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_watten.envs.watten_env.WattenEnv'>' has deprecated methods. Compatibility code invoked.


# Search

In [3]:
def search(obs):
    global sample_outputs, sample_inputs, next_index
    state = env.get_state()
    n = len(env.players[env.current_player].hand_cards)   
    p = 0
    local_index = next_index
    next_index += 1
    sample_inputs[0][local_index] = np.array(obs[0])
    sample_inputs[1][local_index] = np.array(obs[1])
    current_player = env.current_player
    
    for i in range(n):
        card_id = env.players[env.current_player].hand_cards[i].id
        obs, rew, is_done, _ = env.step(card_id)
        
        if is_done:
            sample_outputs[local_index][card_id] = (rew[0] > 0)
        else:
            sample_outputs[local_index][card_id] = ((1 - search(obs)) if current_player != env.current_player else search(obs))
        env.set_state(state)
        p += sample_outputs[local_index][card_id]
        
    return sample_outputs[local_index].max()

In [4]:
def reset_samples(iterations):
    global sample_outputs, sample_inputs,next_index
    sample_outputs = np.full((30000*iterations, 32), 0, dtype=float)
    sample_inputs = [np.zeros((30000*iterations, 4, 8, 2), dtype=int), np.zeros((30000*iterations, 4), dtype=int)]
    next_index = 0
    
def postprocess_samples():
    global sample_outputs, sample_inputs,next_index
    sample_inputs = sample_inputs[:next_index]
    sample_outputs = sample_outputs[:next_index]
    
    #exp = np.exp(sample_outputs)
    #exp[np.where(sample_outputs == -1)] = 0
    #sample_outputs = exp / np.expand_dims(np.sum(exp, axis=-1), -1)
    
def generate_training_data(iterations=1):
    global sample_outputs, sample_inputs,next_index
    reset_samples(iterations)
    
    for i in range(iterations):
        obs = env.reset()
        search(obs)    
        
    postprocess_samples()  
    return sample_inputs, sample_outputs

# Play

In [5]:
def match(agents, render=False):
    obs, is_done = env.reset(), False    
    
    while not is_done:
        input = [np.expand_dims(obs[0], 0), np.expand_dims(obs[1], 0)]
        prediction = agents[env.current_player].predict(input)[0]
        
        valid_moves = np.array([1 if card in env.players[env.current_player].hand_cards else 0 for card in env.cards])       
        prediction_valid = np.ma.masked_where(valid_moves == 0, prediction)
        
        obs, rew, is_done, _ = env.step(np.argmax(prediction_valid))
        
        if render:
            env.render('human')
            sleep(1)
        
    return env.current_player

def compare(agent1, agent2):
    agents = [agent1, agent2]
    first_player_wins = 0
    
    for i in range(1000):
        start_player = random.randint(0, 1)
        winner = match([agents[start_player], agents[1 - start_player]])
        first_player_wins += ((winner == 0) == (start_player == 0))
    
    return first_player_wins / 1000      

# Train

In [7]:
from keras.models import Sequential, clone_model
from keras.layers import Dense, Activation, Input, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, Merge, Flatten, BatchNormalization, add
from keras.layers.merge import concatenate
from keras.models import Model

import keras.backend as K
from IPython.display import clear_output
from keras import optimizers
import math

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
# For a single-input model with 2 classes (binary classification):
def mean_pred(y_true, y_pred):
    return K.mean(K.equal(K.gather(y_true, K.argmax(y_true, axis=-1)), K.gather(y_true, K.argmax(y_pred, axis=-1))))

def acc(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))

def residual_layer(input_block, filters, kernel_size):

    x = conv_layer(input_block, filters, kernel_size)

    x = Conv2D(
    filters = filters
    , kernel_size = kernel_size
    , padding = 'same'
    , use_bias=False
    , activation='linear'
    )(x)

    #x = BatchNormalization(axis=1)(x)

    x = add([input_block, x])

    x = LeakyReLU()(x)

    return (x)

def conv_layer(x, filters, kernel_size):

    x = Conv2D(
    filters = filters
    , kernel_size = kernel_size
    , padding = 'same'
    , use_bias=False
    , activation='linear'
    )(x)

    #x = BatchNormalization(axis=1)(x)
    x = LeakyReLU()(x)

    return (x)
    
input_1 = Input((4,8,2))
convnet = input_1
     
convnet = conv_layer(convnet, 32, (3, 3))
convnet = residual_layer(convnet, 32, (3, 3))
#convnet = Conv2D(32, (3, 3), activation='relu', padding='same')(convnet)
#convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
#convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
#convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
convnet = Flatten()(convnet)

input_2 = Input((4,))

dense_out = concatenate([convnet, input_2])
#dense_out = Dense(128, activation='relu')(dense_out)
#dense_out = Dense(256, activation='relu')(dense_out)
dense_out = Dense(256, activation='relu')(dense_out)
dense_out = Dense(32, activation='sigmoid')(dense_out)

model = Model(inputs=[input_1, input_2], outputs=[dense_out])

adam = optimizers.Adam()
model.compile(optimizer=adam,
              loss='mean_squared_error',
              metrics=['accuracy', acc])


In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 8, 2)      0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 4, 8, 32)     576         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 4, 8, 32)     0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 4, 8, 32)     9216        leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
leaky_re_l

In [26]:
prev_states = []
for i in range(10):
    prev_states.append(clone_model(model))
    prev_states[-1].set_weights(model.get_weights())
    generate_training_data(100)    
    p = np.random.permutation(int(len(sample_outputs) * 0.05))
    sample_inputs, sample_outputs = [sample_inputs[0][p], sample_inputs[1][p]], sample_outputs[p]
    
    # Train the model, iterating on the data in batches of 32 samples
    model.fit(sample_inputs, sample_outputs, epochs=1, batch_size=64)    
    
    print("New model wins " + str([compare(model, prev_state) * 100 for prev_state in prev_states]) + "%!")

Epoch 1/1
75692/75692 [==============================] - 19s 245us/step - loss: 0.0041 - acc_1: 0.4389 - acc_2: 0.4389
New model wins [48.3]%!
Epoch 1/1
73942/73942 [==============================] - 18s 242us/step - loss: 0.0045 - acc_1: 0.4313 - acc_2: 0.4313
New model wins [51.5, 53.400000000000006]%!
Epoch 1/1
75869/75869 [==============================] - 19s 246us/step - loss: 0.0043 - acc_1: 0.4404 - acc_2: 0.4404
New model wins [50.4, 53.7, 49.3]%!
Epoch 1/1
77969/77969 [==============================] - 19s 246us/step - loss: 0.0045 - acc_1: 0.4424 - acc_2: 0.4424
New model wins [47.099999999999994, 50.9, 48.5, 50.1]%!
Epoch 1/1
78731/78731 [==============================] - 20s 253us/step - loss: 0.0046 - acc_1: 0.4495 - acc_2: 0.4495
New model wins [53.2, 51.5, 50.2, 51.6, 51.9]%!
Epoch 1/1
80174/80174 [==============================] - 20s 247us/step - loss: 0.0043 - acc_1: 0.4409 - acc_2: 0.4409
New model wins [49.6, 56.39999999999999, 50.0, 49.3, 52.1, 47.199999999999996]

In [30]:
model.save('model.h5')

In [ ]:
model.load_weights('model.h5')

# Eval

In [ ]:
generate_training_data(100)    
p = np.random.permutation(len(sample_inputs))
sample_inputs, sample_outputs = sample_inputs[p], sample_outputs[p]

In [ ]:
model.evaluate(sample_inputs, sample_outputs)

# Predict

In [12]:
def generate_input(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    obs = [np.zeros([4, 8, 2]), np.zeros([4])]
    
    for card in hand_cards:
        obs[0][card.color.value][card.value.value][0] = 1
        
    if table_card is not None:
        obs[0][table_card.color.value][table_card.value.value][1] = 1

    obs[1][0] = (own_tricks == 1 or own_tricks == 3)
    obs[1][1] = (own_tricks == 2 or own_tricks == 3)

    obs[1][2] = (opponent_tricks == 1 or opponent_tricks == 3)
    obs[1][3] = (opponent_tricks == 2 or opponent_tricks == 3)
    
    return [np.expand_dims(obs[0], 0), np.expand_dims(obs[1], 0)]

In [13]:
def card(color, value):
    for card in env.cards:
        if card.value == value and card.color == color:
            return card
        
    return None

In [14]:
def calc_correct_output_sample(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    global sample_outputs
    env.reset()
    env.cards_left = env.cards[:]
    random.shuffle(env.cards_left)
    
    for card in hand_cards:
        env.cards_left.remove(card)
    env.players[0].hand_cards = hand_cards[:]
        
    env.players[1].hand_cards = []
    for i in range(len(hand_cards) - (1 if table_card is not None else 0)):
        env.players[1].hand_cards.append(env.cards_left.pop())       
    
    env.players[0].tricks = own_tricks
    env.players[1].tricks = opponent_tricks
    env.table_card = table_card
    
    #obs, rew, is_done, _ = env.step(hand_cards[0].id)
   # print(rew, is_done)

    #env.render('human')
    reset_samples(1)
    search(env.regenerate_obs())
    postprocess_samples()
        
    return sample_outputs[0]

In [15]:
def calc_correct_output(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):    
    correct_output = None
    for i in range(100):
        sample_outputs = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks)
        if correct_output is None:
            correct_output = sample_outputs
        else:
            correct_output += sample_outputs
    return correct_output / 100

In [16]:
def predict(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
    output = model.predict(input)[0]
    correct_output = calc_correct_output(hand_cards, table_card, own_tricks, opponent_tricks)
    print(output)
    for card in hand_cards:
        print(str(card.color) + " " + str(card.value) + " => " + str(output[card.id]) + " (" + str(correct_output[card.id]) + ")")

In [17]:
def eval():
    correct = 0
    for i in range(100):
        own_tricks = random.randint(0, 2)
        opponent_tricks = random.randint(0, 2)
        table_card_enabled = random.randint(0, 1)
        hand_cards = random.sample(env.cards, 5 - own_tricks - opponent_tricks + table_card_enabled)
        table_card = hand_cards.pop() if table_card_enabled else None

        input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
        output = model.predict(input)[0]
        correct_output = calc_correct_output(hand_cards, table_card, own_tricks, opponent_tricks)
        
        #print(output, np.argmax(output)) 
        #print(correct_output, np.argmax(correct_output))
        correct += correct_output[np.argmax(output)] == correct_output[np.argmax(correct_output)]
        if correct_output[np.argmax(output)] != correct_output[np.argmax(correct_output)]:
            print(correct_output[np.argmax(output)], correct_output[np.argmax(correct_output)])
    return correct / 100

In [29]:
eval()

0.87 0.9
0.58 0.64
0.52 0.57
0.84 1.0
0.13 0.21
0.06 0.18
0.85 0.88
0.13 0.19
0.97 0.98
0.57 0.63
0.97 1.0
0.66 0.68
0.97 0.99
0.13 0.35
0.26 0.29
0.05 0.1
0.76 0.78
0.08 0.11
0.43 0.45
0.39 0.66
0.05 0.06
0.0 0.02


0.78

In [27]:
predict([
    card(Color.EICHEL, Value.ACHT), 
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SAU)
    ], None, 0, 2)


[1.01765745e-05 1.99414581e-01 5.17120352e-04 9.99587974e-06
 8.74017496e-06 2.65750714e-04 1.74934380e-02 1.68104947e-04
 9.42155384e-06 1.30696004e-04 1.34896909e-04 1.01882215e-04
 1.54784120e-05 1.21838075e-05 9.30142323e-06 2.24108662e-04
 3.77140075e-01 2.42678634e-05 1.59662217e-04 2.49872304e-04
 9.87240201e-05 8.54951853e-04 3.50388815e-04 5.82039502e-05
 2.86130380e-05 2.00224404e-05 4.72779975e-05 6.15803699e-04
 9.47697299e-06 2.31899539e-05 1.30007887e-04 1.57487171e-04]
Color.EICHEL Value.ACHT => 0.017493438 (0.06)
Color.EICHEL Value.KOENIG => 0.19941458 (0.06)
Color.HERZ Value.SAU => 0.37714007 (0.06)


In [28]:
predict([
    card(Color.EICHEL, Value.SAU), 
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SAU)
    ],card(Color.EICHEL, Value.ZEHN), 1, 0)

[9.4750643e-01 9.9227762e-01 5.7221134e-04 3.8952279e-05 5.9414690e-04
 9.4874878e-04 2.0263714e-03 3.9270180e-03 4.2662266e-04 9.7591466e-05
 5.6672929e-05 1.8299887e-03 2.3957381e-03 1.9789851e-04 2.9081915e-04
 5.8614719e-04 8.3646917e-01 1.7605552e-03 2.9425157e-04 2.6143081e-03
 1.5922552e-01 5.3230335e-04 1.0656477e-03 1.3089181e-04 1.8379923e-03
 1.3541707e-03 1.6923452e-03 2.1982193e-04 6.7404872e-05 2.1093113e-03
 8.5485539e-05 2.2106292e-03]
Color.EICHEL Value.SAU => 0.9475064 (0.95)
Color.EICHEL Value.KOENIG => 0.9922776 (0.95)
Color.HERZ Value.SAU => 0.8364692 (0.6)
